In [ ]:
# HIDDEN
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [ ]:
# Some functions for plotting. You don't have to understand how any
# of the functions in this cell work, since they use things we 
# haven't learned about in Data 8.


def resize_window(lim=3.5):
    plots.xlim(-lim, lim)
    plots.ylim(-lim, lim)
    
def draw_line(slope=0, intercept=0, x=make_array(-4, 4), color='r'):
    y = x*slope + intercept
    plots.plot(x, y, color=color)
    
def draw_vertical_line(x_position, color='black'):
    x = make_array(x_position, x_position)
    y = make_array(-4, 4)
    plots.plot(x, y, color=color)
    
def make_correlated_data(r):
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    return x, y

def r_scatter(r):
    """Generate a scatter plot with a correlation approximately r"""
    plots.figure(figsize=(5,5))
    x, y = make_correlated_data(r)
    plots.scatter(x, y, color='darkblue', s=20)
    plots.xlim(-4, 4)
    plots.ylim(-4, 4)
    
def r_table(r):
    """
    Generate a table of 1000 data points with a correlation approximately r
    """
    np.random.seed(8)
    x, y = make_correlated_data(r)
    return Table().with_columns('x', x, 'y', y)

## Correlation ##

In [ ]:
hybrid = Table.read_table('hybrid.csv')
suv = hybrid.where('class', 'SUV').drop('class', 'year')

In [ ]:
suv.scatter('mpg', 'msrp')

In [ ]:
def standard_units(x):
    """Converts an array x to standard units"""
    return (x - np.mean(x)) / np.std(x)

In [ ]:
# Convert MPG and MSRP columns to standard units
# and put them back in the table.

mpg = suv.column('mpg')
msrp = suv.column('msrp')

suv = suv.with_columns(
    'mpg in standard units', standard_units(mpg),
    'msrp in standard units', standard_units(msrp),
)

In [ ]:
def correlation(t, x, y):
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.mean(x_su * y_su)

In [ ]:
suv.scatter('mpg in standard units', 'msrp in standard units')

In [ ]:
suv

### Correlation coefficient: r

In [ ]:
r_scatter(-1)

In [ ]:
r_scatter(0.5)

In [ ]:
r_scatter(-0.5)

In [ ]:
r_scatter(0)

### Nonlinearity ###

In [ ]:
new_x = np.arange(-4, 4.1, 0.5)
nonlinear = Table().with_columns(
        'x', new_x,
        'y', new_x**2
    )
nonlinear.scatter('x', 'y', s=50, color='r')

In [ ]:
correlation(nonlinear, 'x', 'y')

### Outliers ###

In [ ]:
line = Table().with_columns(
        'x', make_array(1, 2, 3, 4),
        'y', make_array(1, 2, 3, 4)
    )
line.scatter('x', 'y', s=50, color='r')

In [ ]:
correlation(line, 'x', 'y')

In [ ]:
outlier = Table().with_columns(
        'x', make_array(1, 2, 3, 4, 5),
        'y', make_array(1, 2, 3, 4, 0)
    )
outlier.scatter('x', 'y', s=50, color='r')

In [ ]:
correlation(outlier, 'x', 'y')

In [ ]:
outlier2 = Table().with_columns(
        'x', make_array(1, 2, 3, 4, 5),
        'y', make_array(1, 2, -2, 4, 5)
    )
outlier2.scatter('x', 'y', s=50, color='r')

In [ ]:
correlation(outlier2, 'x', 'y')

### Ecological correlations

In [ ]:
election16 = Table.read_table('election2016_state.csv')
election16

In [ ]:
election16.scatter('Median Income', 'Percent voting for Clinton')

In [ ]:
party = Table.read_table('party_affiliation2016.csv')
party

In [ ]:
party.exclude(6).barh('Income')

In [ ]:
sat2014 = Table.read_table('sat2014.csv').sort('State')
sat2014.show(4)

In [ ]:
sat2014.scatter('Critical Reading', 'Math')

In [ ]:
correlation(sat2014, 'Critical Reading', 'Math')

## Prediction lines

In [ ]:
example = r_table(0.99)
example.show(3)

In [ ]:
example.scatter('x', 'y')

In [ ]:
example.scatter('x', 'y')
resize_window()

In [ ]:
def nn_prediction_example(x_val):
    """ Predicts y-value for x based on the example table """
    neighbors = example.where('x', are.between(x_val - .25, x_val + .25))
    return np.mean(neighbors.column('y'))
    

In [ ]:
nn_prediction_example(-2.25)

In [ ]:
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))

In [ ]:
example.scatter('x')
resize_window()

In [ ]:
example.scatter('x')
draw_line(slope=1, color='dodgerblue')
resize_window()

In [ ]:
example = r_table(0)
example.scatter('x', 'y')
resize_window()

In [ ]:
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))

In [ ]:
example = example.with_column(
    'Predicted y', example.apply(nn_prediction_example, 'x'))
example.scatter('x')
#draw_line(slope = 0)
resize_window()

In [ ]:
example = r_table(0.5)
example.scatter('x', 'y')
resize_window()

In [ ]:
example = r_table(0.5)
example.scatter('x', 'y')
resize_window()
draw_vertical_line(1.5)
draw_line(slope=1, intercept=0)

In [ ]:
example = example.with_column('Predicted y', example.apply(nn_prediction_example, 'x'))
example.scatter('x')
draw_line(slope=1)
draw_vertical_line(1.5)
resize_window()

In [ ]:
example.scatter('x')
draw_line(slope=1, intercept=0, color='red')
draw_line(slope=0.5, intercept=0, color='dodgerblue')
resize_window()

## Linear regression: defining the line

In [ ]:
# Copy-pasted from above
def standard_units(x):
    """Converts an array x to standard units"""
    return (x - np.mean(x)) / np.std(x)

def correlation(t, x, y):
    x_su = standard_units(t.column(x))
    y_su = standard_units(t.column(y))
    return np.mean(x_su * y_su)


In [ ]:
def slope(t, x, y):
    ...

def intercept(t, x, y):
    ...

In [ ]:
example = r_table(0.5)
slope(example, 'x', 'y')

## Galton height data

In [ ]:
galton = Table.read_table('galton.csv')

heights = Table().with_columns(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight'))
heights

In [ ]:
def nn_prediction_galton(h):
    """Return a prediction of the height of a child 
    whose parents have a midparent height of h.
    
    The prediction is the average height of the children 
    whose midparent height is in the range h plus or minus 0.5 inches.
    """
    neighbors = heights.where(
        'MidParent', are.between(h - 0.5, h + 0.5))
    return np.mean(neighbors.column('Child'))

In [ ]:
heights_with_predictions = heights.with_column(
    'Average neighbor prediction', 
    heights.apply(nn_prediction_galton, 'MidParent'))

In [ ]:
galton_slope = slope(heights, 'MidParent', 'Child')
galton_intercept = intercept(heights, 'MidParent', 'Child')
galton_slope, galton_intercept

In [ ]:
heights.take(123)

In [ ]:
### Make prediction for child height


In [ ]:
heights_with_predictions.where('MidParent', are.equal_to(69.48))

In [ ]:
heights_with_predictions = heights_with_predictions.with_column(
    'Regression Prediction', 
    galton_slope*heights.column('MidParent') + galton_intercept
)
heights_with_predictions

In [ ]:
heights_with_predictions.scatter('MidParent')